# ROOT histograms

[Histogram class documentation](https://root.cern.ch/doc/master/classTH1.html)

ROOT has powerful histogram objects that, among other features, let you produce complex plots and perform fits of arbitrary functions.

Below is an example histogram that can be obtained using a tutorial: [Higgs to Four Leptons](https://root.cern/doc/master/df106__HiggsToFourLeptons_8py.html). 

`TH1F` is a 1D histogram with Floating point y-axis, `TH2I` is a 2D histogram with Integer y-axis, etc.

<center><img src="img/examplehist_df106_HiggsToFourLeptons.png"><center>

# Create a Simple Histogram
builds and draws a histogram with the following features:

- The number of bins is 50 and the x axis ranges from 0 to 10.
- It is filled with random numbers distributed according to an exponential distribution which has a rate = 0.5.
  Suggestion: see the [TRandom](https://root.cern.ch/doc/master/classTRandom.html) for generating random numbers or
  [TH1::FillRandom](https://root.cern.ch/doc/master/classTH1.html#random-numbers)
- Its line width is thicker than the default one.




We now create our histogram

In [1]:
TH1D hist1("myHisto", "Exponential distribution; My axis x; Observed events", 100, 0, 10);

We use the ROOT random number generator, e.g TRandom3
See https://root.cern.ch/doc/master/classTRandom.html for the list of available PRNG in ROOT and the available functions to generate random numbers according to some specific distributions.

Note that the parameter to pass to  `TRandom::Exp` is the exponetial decay parameter, the inverse of the exponential rate.

In [2]:
TRandom3 r(1);
for (int i = 0; i < 10000; i++)
    hist1.Fill(r.Exp(3));

[Drawing options documentation](https://root.cern.ch/doc/master/classTHistPainter.html)

The link above contains the documentation for the histogram drawing options.

In a notebook, we want to use the `%jsroot on` magic and explicitly draw a `TCanvas`.

Time for styling the histogram setting a thicker line and use jsroot

In [3]:
%jsroot on
hist1.SetLineWidth(3);
TCanvas c;
hist1.Draw("");
c.Draw();

We can also change the histogram color to blue.
We don't need to re-draw the histogram. It is enough to call `TCanvas::Draw()`

In [4]:
hist1.SetLineColor(kBlue);
hist1.SetMarkerColor(kBlue);
c.Draw();

In [5]:
TH1D hist2("myHisto2", "", 100, 0, 10); // same bins and x range
TRandom3 r(1);
for (int i = 0; i < 3000; i++)
    hist2.Fill(r.Gaus(5, 1));

hist2.SetLineColor(kRed);
hist2.SetMarkerColor(kRed);
hist2.Draw("same");

c.Draw();


### You can add/divide/scale histograms
Let's create the 3rd histogram which will be just the sum of $hist1 + hist2$ multiplied by 0.5

In [6]:
TH1D *hist3 = (TH1D*)hist1.Clone("myHisto3");

hist3->SetLineColor(kGreen);
hist3->SetMarkerColor(kGreen);
hist3->Add(&h2);
hist3->Scale(0.5);
hist3->Draw("same");

c.Draw();


input_line_76:2:45: error: use of undeclared identifier 'h2'
 (((*(class TH1D **)0x7f94f8052000))->Add(& h2))
                                            ^
Error in <HandleInterpreterException>: Error evaluating expression (((*(class TH1D **)0x7f94f8052000))->Add(& h2))
Execution of your code was aborted.


## Remark - What about errors?

By default, they are counted as $\sqrt{N}$, where $N$ is the number of entries in this particular bin. 

This comes from  the fact that the underlying statistical distribution is a Poisson distribution


By default, for each bin, the sum of weights is computed at fill time. One can also call `TH1::Sumw2` to force the storage and computation of the sum of the square of weights per bin. If `Sumw2` has been called, the error per bin is computed as the $\sqrt{\sum_i{w_i^2}}$, otherwise it is just $\sqrt{N}$.


<center><img src="img/ErrorPropagation.png" alt="drawing" width="500"/></center>


## ROOT graphs

[TGraph](https://root.cern/doc/master/classTGraph.html) is a type useful for scatter plots.

Their drawing options are documented [here](https://root.cern/doc/master/classTGraphPainter.html).

Like for histograms, the aspect of `TGraph`s can be greatly customized, they can be fitted with custom functions, etc. 

Let's create a graph with the following features: 
- The title of the plot is My graph.
- The x and y axis have labels my_{X} and my_{Y} respectively.
- It has three points with the following coordinates (1,0), (2,3), (3,4).
- The marker is a full square. Its coulour is red.
- An orange line joins the points.

We now create the TGraph class 

In [7]:
auto g = new TGraph();

We fill now the Graph with the points

In [8]:
double x[] = {1,2,3};
double y[] = {0,3,4};

In [9]:
for (int i = 0; i < 3; i++)
    g->SetPoint(i,x[i],y[i]);

Before drawing it we set its graphic style

In [10]:
g->SetTitle("My graph;my_{X};my_{Y}");
g->SetLineColor(kOrange);
g->SetLineWidth(2);
g->SetMarkerStyle(kFullSquare);
g->SetMarkerColor(kRed);

We draw the Graph in a ROOT canvas using options: 
- `"A"` to draw the axis
- `"P"` to draw the points
- `"L"` to connect the points with a line

See [TGraphPainter](https://root.cern.ch/doc/master/classTGraphPainter.html) for all the drawing options

In [11]:
auto c = new TCanvas();
g->Draw("APL");
c->Draw();

Change marker style, colour as well as line colour and thickness. Make the plot interactive. Re-draw the plot and interact with it!

In [12]:
g->SetMarkerStyle(kFullCircle);
g->SetMarkerColor(kBlue);
g->SetLineColor(kGreen);

Now we set also the grid on the canvas.

In [13]:
c->SetGrid();
c->Draw();

# ROOT functions

The type that represents an arbitrary one-dimensional mathematical function in ROOT is [TF1](https://root.cern.ch/doc/master/classTF1.html).<br>
Similarly, [TF2](https://root.cern.ch/doc/master/classTF2.html) and [TF3](https://root.cern.ch/doc/master/classTF3.html) represent 2-dimensional and 3-dimensional functions.

## Example of a Function 

- Create a function with formula *cos(x)* and draw it.
- Create another *cos(x)*, but scale the argument of the cosine by adding a parameter.
- Set a value for the parameter.
- Change the line color of the second function.
- Draw the second function in the same canvas as the first one.

We now create the TF1 class with formula *cos(x)*

In [14]:
auto f1 = new TF1("f1", "cos(x)", 0., 10.);

We draw now the function.

In [15]:
f1->Draw();
gPad->Draw();

Create another function based on *cos(x)* with a parameter

In [16]:
auto f2 = new TF1("f2", "cos(x/[0])", 0., 10.);

In [17]:
f2->SetParameter(0, 2);
f2->SetLineColor(kBlue);

We draw the second function in the same Pad as the first one

In [18]:
f2->Draw("SAME");
gPad->Draw();

### Let's add a legend to the plot

In [19]:
gPad->Clear();
f1->Draw();
f2->Draw("SAME");


TLegend leg (0.7, 0.7, 0.9, 0.9);
leg.AddEntry(f1, "cos(x)", "l");
leg.AddEntry(f2, "cos(x/2)", "l");
leg.SetFillColor(0);
leg.Draw();

gPad->Draw();